# Library and imports

In [1]:
import urllib.request
from selenium import webdriver
from selenium.common.exceptions import StaleElementReferenceException, TimeoutException
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium import webdriver
import time
import random
import numpy as np

# Open playlist page and load html

In [2]:
Lex_Podcast_Playlist_Link = "https://www.youtube.com/playlist?list=PLrAXtmErZgOdP_8GztsuKi9nrraNbKKp4"
driver = webdriver.Chrome()
driver.get(Lex_Podcast_Playlist_Link)

# We need to load the full page and extract the html to get the titles, URLs and other details. 
#To do so, we need to scroll 
# through the page when opening it. The following code serves this purpose. 

for i in range(10):
    driver.execute_script('window.scrollTo(0,(window.pageYOffset+3000))')
    time.sleep(1)

Page_Source_HTML = str(driver.page_source)

# Extract video details

This might very well be the worst piece of code you read this week, or even this year, but I couldn't be bothered to look for a more elegant version to do it. I manually looked through the html code and found the strings systematically preceding the video titles, times and urls. From then, I looked for the occurences of these strings and got the details after. If you have a better way to do this, do not hesitate to message me and I will modify it. 

In [3]:
def find_all(a_str, sub):
    start = 0
    while True:
        start = a_str.find(sub, start)
        if start == -1: return
        yield start
        start += len(sub) # use start += 1 to find overlapping matches

Extract videos URLs

In [4]:
string_to_find = str('nline-block style-scope ytd-thumbnail" aria-hidden="true" tabindex="-1" rel="null" href=')
Raw_URLs = find_all(Page_Source_HTML, string_to_find)
URLs = []
for url in Raw_URLs:
    Individual_URL = 'https://www.youtube.com/' + str(Page_Source_HTML[url+90:url+110].split(';')[0])
    URLs.append(Individual_URL)


Extact videos Titles

In [5]:
string_to_find = 'video-title" class="style-scope ytd-playlist-video-renderer" aria-label='
Raw_Titles = find_all(Page_Source_HTML, string_to_find)
Titles = []
for t in Raw_Titles:
    Individual_Title = Page_Source_HTML[t+73:t+200].split('by')[0]
    Titles.append(Individual_Title)

Extract videos Durations

In [6]:
string_to_find = 'ytd-thumbnail-overlay-time-status-renderer" aria-label='
Raw_Durations = find_all(Page_Source_HTML, string_to_find)
Durations = []
for dur in Raw_Durations:
    Individual_Duration = Page_Source_HTML[dur+56:dur+100].split('">')[0]
    Durations.append(Individual_Duration)

Checking everything makes sense

In [7]:
if len(Durations)!= len(URLs) or len(URLs) != len(Titles):
    print('Something is wrong... ')
else:
    print('We have', len(URLs), 'available podcasts by Lex!')
    print('Here are some example for sanity check:\n')
    ran = np.int_(np.random.rand(5,)*100)
    for r in ran:
        print('\nTitle:', Titles[r], ' - Duration:', Durations[r], '- URL:', URLs[r])

We have 140 available podcasts by Lex!
Here are some example for sanity check:


Title: Jim Keller: Moore's Law, Microprocessors, and First Principles | Lex Fridman Podcast #70   - Duration: 1 hour, 34 minutes, 44 seconds - URL: https://www.youtube.com/watch?v=Nb2tebYAaOA&

Title: Elon Musk: Neuralink, AI, Autopilot, and the Pale Blue Dot | Lex Fridman Podcast #49   - Duration: 36 minutes, 10 seconds - URL: https://www.youtube.com/watch?v=smK9dgdTl40&

Title: David Chalmers: The Hard Problem of Consciousness | Lex Fridman Podcast #69   - Duration: 1 hour, 38 minutes, 49 seconds - URL: https://www.youtube.com/watch?v=LW59lMvxmY4&

Title: Brian Kernighan: UNIX, C, AWK, AMPL, and Go Programming | Lex Fridman Podcast #109   - Duration: 1 hour, 43 minutes, 10 seconds - URL: https://www.youtube.com/watch?v=O9upVbGSBFo&

Title: Matt Botvinick: Neuroscience, Psychology, and AI at DeepMind | Lex Fridman Podcast #106   - Duration: 2 hours, 32 seconds - URL: https://www.youtube.com/watch?v=3t06aj

# Save details for later use

In [9]:
import pickle 

URLs_File = open('URLs_Lex_Fridman.pkl', 'wb')
pickle.dump(URLs, URLs_File)
URLs_File.close()

Titles_File = open('Titles_Lex_Fridman.pkl', 'wb')
pickle.dump(Titles, Titles_File)
Titles_File.close()


Durations_File = open('Durations_Lex_Fridman.pkl', 'wb')
pickle.dump(Durations, Durations_File)
Durations_File.close()